In [ ]:
# Enable the webrisk api

In [ ]:
# Submit test


In [1]:
!pip install google-cloud-webrisk

  Obtaining dependency information for google-cloud-webrisk from https://files.pythonhosted.org/packages/aa/7a/038e70b2ceca49d9c03d09a9665f7469fe962e38cff2a6062844613e33a2/google_cloud_webrisk-1.12.2-py2.py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.7/93.7 kB 8.9 MB/s eta 0:00:00


In [9]:
# [START webrisk_search_uri]
from google.cloud import webrisk_v1
from google.cloud.webrisk_v1 import SearchUrisResponse


def search_uri(
    uri: str, threat_type: webrisk_v1.ThreatType.MALWARE
) -> SearchUrisResponse:
    """Checks whether a URI is on a given threatList.

    Multiple threatLists may be searched in a single query. The response will list all
    requested threatLists the URI was found to match. If the URI is not
    found on any of the requested ThreatList an empty response will be returned.

    Args:
        uri: The URI to be checked for matches
            Example: "http://testsafebrowsing.appspot.com/s/malware.html"
        threat_type: The ThreatLists to search in. Multiple ThreatLists may be specified.
            Example: threat_type = webrisk_v1.ThreatType.MALWARE

    Returns:
        SearchUrisResponse that contains a threat_type if the URI is present in the threatList.
    """
    webrisk_client = webrisk_v1.WebRiskServiceClient()

    request = webrisk_v1.SearchUrisRequest()
    request.threat_types = [threat_type]
    request.uri = uri

    response = webrisk_client.search_uris(request)
    if response.threat.threat_types:
        print(f"The URI has the following threat: {response}")
    else:
        print("The URL is safe!")
    return response

uri="http://testsafebrowsing.appspot.com/s/malware.html"

response = search_uri(
            uri,
            webrisk_v1.ThreatType.MALWARE,
    )


The URI has the following threat: threat {
  threat_types: MALWARE
  expire_time {
    seconds: 1704744563
    nanos: 188586121
  }
}



In [2]:
from google.cloud import webrisk_v1
from google.cloud.webrisk_v1 import Submission


def submit_uri(project_id: str, uri: str) -> Submission:
    """Submits a URI suspected of containing malicious content to be reviewed.

    Returns a google.longrunning.Operation which, once the review is complete, is updated with its result.
    You can use the [Pub/Sub API] (https://cloud.google.com/pubsub) to receive notifications for the
    returned Operation.
    If the result verifies the existence of malicious content, the site will be added to the
    Google's Social Engineering lists in order to protect users that could get exposed to this
    threat in the future. Only allow-listed projects can use this method during Early Access.

     Args:
         project_id: The name of the project that is making the submission.
         uri: The URI that is being reported for malicious content to be analyzed.
             uri = "http://testsafebrowsing.appspot.com/s/malware.html"

    Returns:
        Submission response that contains the URI submitted.
    """
    webrisk_client = webrisk_v1.WebRiskServiceClient()

    # Set the URI to be submitted.
    submission = webrisk_v1.Submission()
    submission.uri = uri

    # Confidence that a URI is unsafe.
    threat_confidence = webrisk_v1.ThreatInfo.Confidence(
        level=webrisk_v1.ThreatInfo.Confidence.ConfidenceLevel.MEDIUM
    )

    # Context about why the URI is unsafe.
    threat_justification = webrisk_v1.ThreatInfo.ThreatJustification(
        # Labels that explain how the URI was classified.
        labels=[
            webrisk_v1.ThreatInfo.ThreatJustification.JustificationLabel.AUTOMATED_REPORT
        ],
        # Free-form context on why this URI is unsafe.
        comments=["Testing submission"],
    )

    # Set the context about the submission including the type of abuse found on the URI and
    # supporting details.
    threat_info = webrisk_v1.ThreatInfo(
        # The abuse type found on the URI.
        abuse_type=webrisk_v1.types.ThreatType.SOCIAL_ENGINEERING,
        threat_confidence=threat_confidence,
        threat_justification=threat_justification,
    )

    # Set the details about how the threat was discovered.
    threat_discovery = webrisk_v1.ThreatDiscovery(
        # Platform on which the threat was discovered.
        platform=webrisk_v1.ThreatDiscovery.Platform.MACOS,
        # CLDR region code of the countries/regions the URI poses a threat ordered
        # from most impact to least impact. Example: "US" for United States.
        region_codes=["US"],
    )

    request = webrisk_v1.SubmitUriRequest(
        parent=f"projects/{project_id}",
        submission=submission,
        threat_info=threat_info,
        threat_discovery=threat_discovery,
    )

    response = webrisk_client.submit_uri(request).result(timeout=30)
    return response


In [3]:
uri="http://testsafebrowsing.appspot.com/s/malware.html"
submit_uri('genai-embeddings',uri)

MethodNotImplemented: 501 Method not found.